In [1]:
# Logistic Regression Classification

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
from Bio.SeqRecord import SeqRecord
from Bio import SeqUtils

In [2]:
# Importing the dataset
dataset = pd.read_csv('final file hopefully no errors.csv')

In [3]:
#for fib2.cvs only

dataset.loc[dataset['Gen'] == 'ssRNA(+)', 'Gen'] = 1
dataset.loc[dataset['Gen'] == 'ssRNA(-)', 'Gen'] = 0

In [4]:
ps = dataset.loc[dataset['Gen'] == 1].reset_index()
ns = dataset.loc[dataset['Gen'] == 0].reset_index()

In [5]:
pn = pd.concat([ps,ns]).reset_index(drop = True)

In [6]:
res = pd.read_csv('res 340.csv')

In [7]:
r = res[['ATC', 'ATCA', 'ATCT', 'TGGT']].copy()

In [8]:
pnr = pd.concat([pn, r], axis = 1)

In [9]:
pnr = pnr.drop(index = pnr.loc[pnr['MolTyp'] == 'DNA'].index)

In [10]:
pnr = pnr.drop(columns=['index', 'Id', 'MolTyp', 'Topol', 'Des'])

In [11]:
pnr = pnr.rename(columns={'ATC' : 'E1', 'ATCA' : 'E2', 'ATCT' : 'E3', 'TGGT' : 'E4'})

sample

In [12]:
ps = pnr.loc[pnr['Gen'] == 1]
ns = pnr.loc[pnr['Gen'] == 0]

pnrs = pd.concat([ps,ns])

make features

In [13]:
#feature 1: length
pnrs['F1'] = [len(i) for i in pnrs['Seq'].to_list()]

In [14]:
#orf_translator function to find orfs and translate them

#a = Seq(pnrs.loc[0,'Seq'], IUPAC.unambiguous_dna)
#rec = SeqRecord(a)
#table = 11
#min_pro_len = 20

def orf_translator(record, table, min_pro_len):
    s1 = []
    s2 = []
    for strand, nuc in [(+1, record.seq), (-1, record.seq.reverse_complement())]:
        for frame in range(3):
            length = 3 * ((len(record)-frame) // 3) #Multiple of three
            for pro in nuc[frame:frame+length].translate(table).split("*"):
                if len(pro) >= min_pro_len:
                    if strand == 1:
                        s1.append(pro)
                    else:
                        s2.append(pro)
    return s1,s2

#orf_translator(rec, 11, 20)

In [15]:
#feature 2: number of orfs on strand 1
#feature 3: number of orfs in strand 2
#feature 4: mean protein length on strand 1
#feature 5: mean protein length on strand 2
#feature 6: GC content
#feature 7: no idea what this feature is

f2, f3, f4, f5, f6, f7 = [],[],[],[],[],[]

for i in pnrs['Seq'].to_list():
    a = SeqRecord(Seq(i))
    (s1, s2) = orf_translator(a, 11, 15)
    f2.append(len(s1))
    f3.append(len(s2))
    
pnrs['F2'] = f2
pnrs['F3'] = f3

In [16]:
#Splitting features and labels
X = pnrs.iloc[:, 3:].values.astype(float)
y = pnrs.iloc[:, 1].values.astype(float)

In [17]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [18]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [19]:
### Training the Logistic Regression model on the Training set
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [20]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [21]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)


In [22]:
cm

array([[ 62,  11],
       [ 11, 701]])

In [23]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.85      0.85      0.85        73
         1.0       0.98      0.98      0.98       712

    accuracy                           0.97       785
   macro avg       0.92      0.92      0.92       785
weighted avg       0.97      0.97      0.97       785



In [25]:
pnrs.describe()

,E1,E2,E3,E4,F1,F2,F3
count,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000
mean,0.030904,0.010100,0.008423,0.010541,10988.525175,162.128107,232.636711
std,0.004766,0.002466,0.001843,0.002493,7183.259646,96.097606,158.159160
min,0.017545,0.003667,0.003266,0.002627,1183.000000,18.000000,24.000000
25%,0.028220,0.008728,0.007427,0.009357,7417.000000,112.000000,149.000000
50%,0.029582,0.009667,0.008139,0.010283,7560.000000,120.000000,164.000000
75%,0.032595,0.011095,0.008983,0.011749,11089.500000,173.750000,233.000000
max,0.054656,0.023478,0.020273,0.020584,41178.000000,482.000000,927.000000
